# Fisher's sharp null vs Nayman's population average effect

While reading the mixtape causal inference book, my friends and I were debating about the implications of Fisher's sharp null vs Naymans's population average effect. Under Fisher's sharp null, each treatment unit is assumed to have zero treatment effect. Nayman, however, stated a more general null hypothesis, where the population average effect is assumed to be zero, but each unit could have a treatment effect. 

Based on my reading about their exchanges, both statastician agree that the population average effect is what we are really interested in real life:
- Dr. Neyman: So long as the average yields of any treatments are identical, the question as to whether these treatments affect separate yields on single plots seems to be uninteresting and academic, and certainly I did not consider methods for its solution.
- Professor Fisher: It may be foolish, but that is what the z test was designed for, and the only purpose for which it has been used.

The sharp null hypothesis also allows for permutation test, because unobserved potential outcomes can be assumed to be the same as the observed outcome under the sharp null hypothesis. 

Here I want to test whether permutation tests and t-tests (simple population difference) would give different test results given eitehr Fisher's or Nayman's null hypotheses.

In [1]:
import numpy as np
from scipy import stats

norm = np.random.normal
binom = np.random.binomial
choice = np.random.choice
shuffle = np.random.shuffle

## Population generation

In [2]:
N = 100 # Population size
trmt_ratio = 100 # scale of unit treatment effects
Y0 = 0 # Baseline population mean of y with no treatment 
t = 0 # Average population treatment effects 

xe = norm(size = N) # baseline population mean of x
x_fisher = [0 for i in range(N)] # no unit treatment effect
x_neyman = norm(size=N) * trmt_ratio # unit treatment effect

In [3]:
def y_process(xe, x, t, Y0):
    '''Returns an np.array of population characteristics. 
       
    The array including the xe, x and the potential outcomes of untreated and treated units
    
    Args:
        x (list): Treatment effects of the units
        t (float): Average population treatment effects
        Y0 (float): Baseline population mean of y with no treatment 
    '''
    assert len(x) == len(xe), "xe and x needs to be of the same length"
    
    n = len(xe)
    y0 = xe + Y0
    y1 = y0 + x + t
    
    pop = np.stack([xe, x, y0, y1], axis = 1)
    
    return pop

In [4]:
pop_fisher = y_process(xe, x_fisher, t, Y0)
pop_neyman = y_process(xe, x_neyman, t, Y0)

In [5]:
def pop_generation(N = 100, trmt_ratio = 100, Y0 = 0, t = 0):
    xe = norm(size = N) # baseline population mean of x
    x_fisher = [0 for i in range(N)] # no unit treatment effect
    x_neyman = norm(size=N) * trmt_ratio # unit treatment effect
    
    pop_fisher = y_process(xe, x_fisher, t, Y0)
    pop_neyman = y_process(xe, x_neyman, t, Y0)
    
    return pop_fisher, pop_neyman

In [6]:
len(pop_fisher)

100

# Sample generation

In [7]:
def sampling(pop, n, e_scale = 1):
    '''Samples from a population <pop> and applies treatment.
    
    The columns of the array are: 
    xe, x, y0, y1, trmt, y without error, error, and observed y
    
    Args:
        pop (np.array): Population characteristics
        n (int): Sample size
        e_scale (float): Sigma of the observation variace
    '''
    col = pop.shape[1]
    sample = np.zeros((n, col + 4))
    
    idx = choice(range(len(pop)), n, replace = False)
    sample[:, :col] = pop[idx, :] # samples a subset of the population
    
    sample[:, col] = binom(n = 1, p = p_trmt, size = n) # treatment
    
    sample[:, col+1] = -1 * sample[:, col - 2] * (sample[:, col] - 1) \
        + sample[:, col - 1] * (sample[:, col])\
    
    sample[:, col + 2] = norm(size = n, scale = e_scale)
    sample[:, col + 3] = sample[:, col + 1] + sample[:, col + 2]
    
    return sample

In [8]:
p_trmt = 0.5 # percent receiving treatment
n = 30
sample_fisher = sampling(pop_fisher, n)
sample_neyman = sampling(pop_neyman, n)

In [9]:
#sample_neyman[:5]

## Test treatment effect using the sample with t_test

In [10]:
ya = sample_fisher[sample_fisher[:,4] == 0, -1]
yb = sample_fisher[sample_fisher[:,4] == 1, -1]
tStat, pValue = stats.ttest_ind(ya, yb, equal_var = False) #run independent sample T-Test

In [11]:
tStat, pValue

(0.07430396069192363, 0.9413100700316837)

## Test treatment effect using the sample with permutation

In [12]:
def one_perm(sample, shuffle = True):
    '''Calculate trmt effect for one random set'''
    n = len(sample)
    if shuffle:
        shuffled_trmt = choice(sample[:,4], n, replace = False)
    else: 
        shuffled_trmt = sample[:,4]
    ya = sample[shuffled_trmt == 0, -1].mean()
    yb = sample[shuffled_trmt == 1, -1].mean()
    
    return yb - ya

def permutt(sample, perm_n, sample_effect = False):
    '''Estimate p_value using n permutations'''
    perms = [one_perm(sample) for i in range(perm_n)]
    if not sample_effect:
        sample_effect = one_perm(sample, shuffle = False)
    ne = sum([1 for i in perms if i > sample_effect ]) 
    # number of times permutated differences are as extreme as the observed difference
    return ne/perm_n

In [13]:
one_perm(sample_fisher, shuffle = False) # observed difference

-0.032634999273354476

In [14]:
# permutt(sample_fisher, 20)

## Comparing t-test and Permutation test

In [15]:
def get_p(sample, perm_n, **kwargs):
    # get t_test p-value:
    ya = sample[sample[:,4] == 0, -1]
    yb = sample[sample[:,4] == 1, -1]
    _, ttest_p = stats.ttest_ind(ya, yb, equal_var = False) 
    
    sample_effect = yb.mean() - ya.mean()
    perm_p = permutt(sample, perm_n, sample_effect, **kwargs)
    
    return [sample_effect, ttest_p, perm_p]

In [16]:
# get_p(sample_fisher, 20)

## Conduct many samples

In [17]:
def pop_trmt_effect(pop):
    '''Calculate the true treatment effect in a population'''
    return pop[:,3].mean() - pop[:,2].mean()

In [18]:
pop_fisher, pop_neyman = pop_generation(N = 50000, trmt_ratio = 10, Y0 = 0, t = 0)
print("True treatment effect in the population")
print(pop_trmt_effect(pop_fisher), pop_trmt_effect(pop_neyman))

True treatment effect in the population
0.0 0.03054047140762021


In [19]:
sample_fisher = sampling(pop_fisher, n, e_scale = 1)
sample_neyman = sampling(pop_neyman, n, e_scale = 1)
p_fisher = get_p(sample_fisher, 100)
p_neyman = get_p(sample_neyman, 100)

In [20]:
p_fisher

[0.906828020841851, 0.11126327107902693, 0.1]

In [21]:
p_neyman

[-1.1253924221191811, 0.6779955906576554, 1.3]

In [22]:
p_fishers = []
p_neymans = []
for i in range(1000):
    sample_fisher = sampling(pop_fisher, n, e_scale = 1)
    sample_neyman = sampling(pop_neyman, n, e_scale = 1)
    p_fishers.append(get_p(sample_fisher, 500))
    p_neymans.append(get_p(sample_neyman, 500))

In [23]:
import pandas as pd
import seaborn as sns
data_fishers = pd.DataFrame(p_fishers, columns = ["sample_effect", "t-test_p", "permutation_r"])
data_fishers['null'] = "Fisher"
data_neymans = pd.DataFrame(p_neymans, columns = ["sample_effect", "t-test_p", "permutation_r"])
data_neymans['null'] = "Neyman"
data = pd.concat([data_fishers, data_neymans])

Matplotlib is building the font cache; this may take a moment.


In [24]:
data.to_csv("fisher_vs_neyman.csv")

In [25]:
#sns.kdeplot(data = data[['sample_effect', 'color']], x = "sample_effect", fill = "color")